In [7]:
import pandas as pd

# Create a DataFrame
data = pd.read_csv('./data/cleaned_data.csv')
df = pd.DataFrame(data=data)

# Display the first few rows of the DataFrame
df.head()

,SampleID,Actinobacteria,Bacteroides,Bacteroidetes,Bifidobacterium,Clostridium,Cyanobacteria,Firmicutes,Fusobacteria,Lactobacillus,Other,Prevotella,Proteobacteria,Tenericutes,Verrucomicrobia,Fiber_intake_g,Fat_intake_g,Iron_intake_mg,Serum_iron_ug
0,000007117.1075649,0.3244,0.0013,0.0315,0.0000,0.0013,0.0352,0.2629,0.0082,0.0027,0.0461,0.0028,0.2821,0.0010,0.0005,20.37,81.73,13.50,80.03
1,000007115.1075661,0.0347,0.0087,0.1735,0.0018,0.0149,0.0289,0.1783,0.0032,0.0166,0.0153,0.0025,0.5196,0.0000,0.0020,18.69,59.38,19.44,77.60
2,000007123.1075697,0.0493,0.0000,0.0153,0.0000,0.0002,0.1265,0.0451,0.0025,0.1222,0.0024,0.0007,0.6347,0.0000,0.0011,20.27,62.60,19.03,122.20
3,000009713.1130401,0.4052,0.0010,0.0091,0.0000,0.0002,0.0265,0.4219,0.0026,0.0015,0.0055,0.0062,0.1191,0.0000,0.0012,0.20,105.56,16.23,93.48
4,000005598.1130569,0.2394,0.0072,0.0401,0.0001,0.0010,0.0255,0.2582,0.0013,0.0170,0.0209,0.0024,0.3828,0.0003,0.0038,16.00,96.82,14.17,70.19


In [ ]:
from sklearn.model_selection import train_test_split

target_column = 'Serum_iron_ug'

# Features and target
X = df.drop(columns=[target_column, "SampleID"])
y = df[target_column]

# Split the dataset into training and testing sets
# X_holdout will be our hold-out set for final evaluation
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42)

# Model evaluation and selection

On the training set:

- Do k-fold Cross-Validation
- Set up a pipeline with scaling
- Compare models
- Tune hyperparameters
- Use chosen metric to select model


Based on which model performs the best under our chosen metric (e.g. MSE, MAE...) we will then pick a model.

In [ ]:
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import BayesianRidge
from sklearn.pipeline import make_pipeline
import numpy as np

# Cross-validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)
coef_list = []

scoring = {
    'R2': 'r2',
    'MSE': 'neg_mean_squared_error',
    'MAE': make_scorer(mean_absolute_error)
}

# Define the pipeline
# TODO: compare different models: pick a varied selection from:
# linear regression, ridge, lasso, SVR, random forest, gradient boost...
pipeline = make_pipeline(StandardScaler(), BayesianRidge())

# Train on each fold and store coefficients
for train_idx, valid_idx in kf.split(X_train):
    X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
    X_valid_fold, y_valid_fold = X_train.iloc[valid_idx], y_train.iloc[valid_idx]

    pipeline.fit(X_train_fold, y_train_fold)

    model = pipeline.named_steps["bayesianridge"]
    coef_list.append(model.coef_)

# Cross-validation on the whole data
# This is sort of redundant because we're doing CV twice rn, but this version gives
# us the metrics without needing to code it ourselves in the loop above, and above we
# are getting the coefficients, which cross_validate() doesn't give us
cv_results = cross_validate(
    pipeline,
    X_train,
    y_train,
    cv=kf,
    scoring=scoring,
    return_train_score=True
)

print("Mean R²:", np.mean(cv_results['test_R2']))
print("Mean MSE:", -np.mean(cv_results['test_MSE']))  # negate because sklearn returns negative MSE
print("Mean MAE:", np.mean(cv_results['test_MAE']))


# Convert to DataFrame
coef_array = np.array(coef_list)
coef_df = pd.DataFrame({
    "Feature": X.columns,
    "Mean_Coefficient": coef_array.mean(axis=0),
    "Std_Coefficient": coef_array.std(axis=0)
}).sort_values(by="Mean_Coefficient", key=abs, ascending=False)


Mean R²: 0.6151741061807406
Mean MSE: 542.0545610191109
Mean MAE: 18.534923418945016


## Final model evaluation

Once we have picked our model, we fit it on the full training set (X_train) and evaluate on the holdout data to simulate how our model might work on real, unseen data

In [ ]:
# Here we evaluate our model on X_train, X_holdout, y_train, y_holdout

## Simulation tests

Now we have locked in our model, let's stress test it under different conditions.

1. Small data set size
2. Different underlying relationships in the data (e.g. serum iron only affected by age and sex, or serum iron only affected by microbiome, not diet)
3. Lower signal-to-noise ratio (more noise!)

Does the model we picked still perform well in these sub-optimal circumstances? What does it tell us about how our model might work on real world data?